# Лабораторная работа 3. Формирование счёта на оплату услуг

Вариант 13

Лазарев Эдуард Артемович
N33471

Импортируем библиотеки, которые мы будем использовать для генерации pdf.

In [1]:
from pathlib import Path
from weasyprint import HTML

Объявлем шаблон html документа в который будут подставляться значения.

In [2]:
HTML_TEMPLATE = """<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Document</title>
        <style>
            table, th, td {{
                border: 1px solid black;
            }}
            table {{
                border-collapse: collapse;
            }}
        </style>
    </head>
    <body>
        <table style="width: 100%;">
            <tbody>
                <tr>
                    <td rowspan=2 colspan=2>{bank}<br><br>Банк получателя</td>
                    <td>БИК</td>
                    <td rowspan=2>{bik}<br><br>{sch1}</td>
                </tr>
                <tr>
                    <td>Сч. №</td>
                </tr>
                <tr>
                    <td>ИНН {inn}</td>
                    <td>КПП {kpp}</td>
                    <td rowspan=2>Сч. №</td>
                    <td rowspan=2>{sch2}</td>
                </tr>
                <tr>
                    <td colspan=2>{poluch}<br><br>Получатель</td>
                </tr>
            </tbody>
        </table>
        <h2>Счет на оплату № {num} от {date}.</h2>
        <hr>
        <table style="border: none;">
            <tbody>
                <tr style="border: none;">
                    <td style="border: none;">Поставщик<br>(Исполнитель):</td>
                    <th style="border: none;">{post}</th>
                </tr>
                <tr>
                    <td style="border: 5px solid white;"></td>
                    <td style="border: 5px solid white;"></td>
                </tr>
                <tr>
                    <td style="border: none;">Покупатель<br>(Заказчик):</td>
                    <th style="border: none;">{pokup}</th>
                </tr>
                <tr>
                    <td style="border: 5px solid white;"></td>
                    <td style="border: 5px solid white;"></td>
                </tr>
                <tr>
                    <td style="border: none;">Основание:</td>
                    <th style="border: none;">{osnov}</th>
                </tr>
            </tbody>
        </table>
        <br>
        <table style="width: 100%;">
            <tbody>
                <tr>
                    <th>№</th>
                    <th>Товары (работы, услуги)</th>
                    <th>Кол-во</th>
                    <th>Ед.</th>
                    <th>Цена</th>
                    <th>Сумма</th>
                </tr>
                {tab2}
            </tbody>
        </table>
        <table style="border: none; float: right;">
            <tbody>
                <tr style="border: none;">
                    <th style="border: none; float: right">Итого:</th>
                    <td style="border: none;">{itog:.2f}</td>
                </tr>
                <tr>
                    <th style="border: none; float: right">В том числе НДС:</th>
                    <td style="border: none;">{nds:.2f}</td>
                </tr>
                <tr>
                    <th style="border: none; float: right">Всего к оплате:</th>
                    <td style="border: none;">{alll:.2f}</td>
                </tr>
            </tbody>
        </table>
        <br><br><br><br>
        Всего наименований {count}, на сумму {summ:.2f} руб.
        <br><br><br>
        Внимание! 
        <br>
        Оплата данного счета означает согласие с условиями поставки товара. 
        <br>
        Уведомление об оплате обязательно, в противном случае не гарантируется начисление товара на складе. Товар отпускается по факту прихода денег на р/с Поставщика, самовывозом, при наличии доверенности и паспорта.
        <hr>
        <div style="white-space: pre;">Руководитель {rukovod} Бухгалтер {buh}</div>
    </body>
</html>
"""

Объявим функцию, принимающую данные для формирования счёта и генерирующую ПДФ.

In [3]:
def makepdf(bank, bik, sch1, inn, kpp, poluch, sch2,
            num, date, post, pokup, osnov, data, rukovod,
            buh, path):
    s_table = "".join(
        ["<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{:.2f}</td><td>{:.2f}</td></tr>".format(c, *i) 
         for c, i in enumerate(data, 1)])
    itog_count = lambda x: sum(i[4] for i in x)
    itog = itog_count(data)
    count = len(data)
    nds = itog/100*20
    html = HTML_TEMPLATE.format(bank=bank, bik=bik, sch1=sch1, inn=inn,
                                kpp=kpp, poluch=poluch, sch2=sch2, num=num,
                                date=date, post=post, pokup=pokup, osnov=osnov,
                                tab2=s_table, itog=itog, nds=nds, alll=itog,
                                count=count, summ=itog, rukovod=rukovod.rjust(22, "_"),
                                buh=buh.rjust(22, "_"))
    htmldoc = HTML(string=html, 
                   base_url="")
    return Path(path).write_bytes(htmldoc.write_pdf())

Соберём данные для выставления счёта в один словарь.

In [4]:
bill_data = {
    "bank": "АО \"Банк\" г. Москва",
    "bik": "111111111",
    "sch1": "11111111111111111111",
    "inn": "1111111111",
    "kpp": "111111111",
    "poluch": "ООО \"Платильшик\"",
    "sch2": "11111111111111111111",
    "num": "1",
    "date": "01 мая 2021 г",
    "post": ("ООО \"Поставщик\", ИНН 1111111111, "
             "КПП 111111111, 111111, г. Москва, "
             "ул. Улица, дом № 1, "
             "тел.: +71111111111"),
    "pokup": ("ООО \"Потребитель\", ИНН 1111111111, "
              "КПП 111111111, 111111, г. Казань, "
              "ул. Улица, дом № 1"),
    "osnov": "№ 11111111 от 30.04.2021",
    "data": [
                ['Телефония', '', '1', 25, 25],
                ['Интернет', '', '1', 5.26, 5.26]
            ],
    "rukovod": "Васильев В.В.",
    "buh": "Володин В.В.",
}

Вызовем функцию для генерации pdf, передав в неё данные для выставления счёта.

In [5]:
makepdf(**bill_data, path="bill.pdf")

32659